# Parameter curation for LDBC SNB Interactive

The goal of Parameter Curation with the goal of selecting parameter bindings that have consistently low-variance intermediate query result sizes throughout the query plan.

In [1]:
import duckdb
import pandas as pd

## LDBC Query1

Parameters:
- PersonId
- firstName

*Given a start Person, find Persons with a given first name (firstName) that the start Person is connected to (excluding start Person) by at most 3 steps via the knows relationships. Return Persons,
including the distance (1..3), summaries of the Persons workplaces and places of study.*



In [2]:
person_firstname = 'personFirstNames'
person_numFriends = 'personNumFriends'
person_numFriendsOfFriends = 'personNumFriendsOfFriends'
person_firstname_path = f"factors/{person_firstname}/*.parquet"
person_person_numFriends = f"factors/{person_numFriends}/*.parquet"
person_numFriendsOfFriends_path = f"factors/{person_numFriendsOfFriends}/*.parquet"


cursor = duckdb.connect() #in-memory db
cursor.execute(f"CREATE VIEW {person_firstname} AS SELECT * FROM read_parquet('{person_firstname_path}');")
cursor.execute(f"CREATE VIEW {person_numFriendsOfFriends} AS SELECT * FROM read_parquet('{person_numFriendsOfFriends_path}');")
cursor.execute(f"CREATE VIEW {person_numFriends} AS SELECT * FROM read_parquet('{person_person_numFriends}');")

In [4]:
cursor.execute(f"CREATE VIEW Person_knows_Person AS SELECT * FROM read_parquet('/home/gladap/repos/ldbc_snb_datagen_spark/out-sf1/graphs/parquet/raw/composite-merged-fk/dynamic/Person_knows_Person/Person_knows_Person.parquet');")

In [5]:
cursor.execute(f"SELECT * FROM Person_knows_Person LIMIT 10").fetch_df().head()

,creationDate,deletionDate,explicitlyDeleted,Person1Id,Person2Id
0,1343643638216,1577664000000,False,332,2866
1,1273127889470,1577664000000,False,332,2869
2,1283862723169,1577664000000,False,332,2199023260815
3,1338430450851,1577664000000,False,332,10995116284819
4,1348248605908,1577664000000,False,332,15393162796516


In [31]:
cursor.execute(f"SELECT * FROM {person_numFriendsOfFriends} LIMIT 10").fetch_df().head()

,Person1Id,creationDate,deletionDate,numFriends,numFriendsOfFriends
0,4398046519825,2010-05-23 13:24:43.393,2019-12-30,977,59259
1,28587302326226,2012-04-11 16:29:23.588,2019-12-30,971,57875
2,37383395346298,2012-11-12 06:12:06.316,2019-12-30,952,56244
3,32985348840714,2012-07-23 00:06:08.096,2019-12-30,947,54189
4,15393162799448,2011-04-13 19:11:44.471,2019-12-30,936,54062


In [65]:
cursor.execute(f"SELECT * FROM {person_numFriends} LIMIT 10").fetch_df().head()

,id,creationDate,deletionDate,frequency
0,4398046519825,2010-05-23 13:24:43.393,2019-12-30,977
1,28587302326226,2012-04-11 16:29:23.588,2019-12-30,971
2,37383395346298,2012-11-12 06:12:06.316,2019-12-30,952
3,32985348840714,2012-07-23 00:06:08.096,2019-12-30,947
4,15393162799448,2011-04-13 19:11:44.471,2019-12-30,936


## Parameter curation 2-hop friends
Select the `percentile_disc` of the number of friends of friends. This is done by using the column `numFriendsOfFriends`.

In [67]:
second_hop_param = cursor.execute(
"""
SELECT
    personId AS 'personId',
    creationDay AS 'maxDate'
FROM
    (SELECT
        id AS personId,
        abs(frequency - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    WHERE frequency > 0 AND deletionDate > '2019' AND creationDate < '2012-11-29'
    ORDER BY diff, md5(id)
    LIMIT 50
    ),
    (SELECT
        creationDay,
        abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM creationDayNumMessages)) AS diff
    FROM creationDayNumMessages
    ORDER BY diff, md5(creationDay)
    LIMIT 20
    )
ORDER BY md5(concat(personId, creationDay))
LIMIT 500

"""
).fetch_df()

RuntimeError: Catalog Error: Table with name creationDayNumMessages does not exist!
Did you mean "personNumFriends"?

In [64]:
second_hop_param.head()

,personId,numFriendsOfFriends,diff
0,28587302324802,8187,0
1,19791209304994,8187,0
2,17592186051125,8188,1
3,13194139540257,8188,1
4,13194139537379,8189,2


In [ ]:
cursor.execute(f"CREATE VIEW Person AS SELECT * FROM read_parquet('/home/gladap/repos/ldbc_snb_datagen_spark/out-sf1/graphs/parquet/raw/composite-merged-fk/dynamic/Person/*.parquet');")


In [49]:
cursor.execute("""
SELECT Person1Id
     , numFriendsOfFriends
   --  , abs(numFriendsOfFriends - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY numFriendsOfFriends) FROM personNumFriendsOfFriends)) AS diff 
  FROM personNumFriendsOfFriends
 WHERE deletionDate < '2019'
 ORDER BY Person1Id
""").fetch_df().head()

,Person1Id,numFriendsOfFriends
0,210,NaN
1,1424,1583.0
2,1953,NaN
3,2238,1366.0
4,2358,NaN


In [16]:
# This is how the personIds are now selected, only based on the first person num friends.
first_hop = cursor.execute("""
               SELECT
        id AS personId,
        abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    ORDER BY diff, md5(id)
    LIMIT 150
               
               """).fetch_df()

In [17]:
first_hop.head()

,personId,diff
0,6597069773053,0
1,15393162792851,0
2,4398046520134,0
3,30786325583816,0
4,35184372091851,0


In [ ]:
for selected_id in [13194139540984]:#first_hop['personId'].values:
    total_friends1 = cursor.execute(f"SELECT frequency FROM personNumFriends WHERE id = {selected_id}").fetchone()[0]
    friends_2 = cursor.execute(f"SELECT Person2Id FROM Person_knows_Person WHERE Person1Id = {selected_id}").fetchall()
    all_friends = friends_1
    print(all_friends)
    total_friends2 = 0
    for friend in all_friends:
        total_friends2 = total_friends2 + cursor.execute(f"SELECT frequency FROM personNumFriends WHERE id = {friend[0]}").fetchone()[0]
    print(len(all_friends))
    total_friends3 = 0
    
    for friend in all_friends:
        friends_4 = cursor.execute(f"SELECT Person2Id FROM Person_knows_Person WHERE Person1Id = {friend[0]} AND Person2Id != {selected_id}").fetchall()
        all_friends2 = friends_4
        for friend2 in all_friends2:
            total_friends3 = total_friends3 + cursor.execute(f"SELECT frequency FROM personNumFriends WHERE id = {friend2[0]}").fetchone()[0]
#         print(f"ID: {selected_id}. Total first hop {total_friends1}. Total friends second hop: {total_friends2}")
    print(f"ID: {selected_id}. Total first hop {total_friends1}. Total friends second hop: {total_friends2}. Total third hop {total_friends3}")


In [ ]:
first_hop = cursor.execute("""
    SELECT count(p2.Person2Id)
    FROM
        Person_knows_Person p1,
        Person_knows_Person p2,
        (
            SELECT
                id AS personId,
                frequency,
                abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
            FROM personNumFriends
            ORDER BY diff, md5(id)
            LIMIT 150
        ) AS first_hop,
     WHERE (p1.Person1Id = first_hop.personId OR p1.Person2Id = first_hop.personId) AND 
     LIMIT 150
                """).fetch_df()

In [ ]:
first_hop.head(150)

In [ ]:
friends1 = cursor.execute("""
        SELECT Person2Id
        FROM Person_knows_Person
        WHERE Person1Id = 13194139540984
        UNION ALL
        SELECT k2.Person2Id
        FROM Person_knows_Person k1, Person_knows_Person k2
        WHERE k1.Person1Id = 13194139540984
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> 13194139540984
    
""").fetch_df()

In [ ]:
len(friends1)

In [ ]:
friends2 = cursor.execute("""
        SELECT k3.Person2Id
        FROM Person_knows_Person k1, Person_knows_Person k2, Person_knows_Person k3
        WHERE k1.Person1Id = 13194139540984
          AND k2.Person1Id = k1.Person2Id
          AND k2.Person2Id = k3.Person1Id
          AND k2.Person2Id <> 13194139540984
          AND k3.Person2Id != 13194139540984
""").fetch_df()

In [ ]:
all_friends = cursor.execute(
"""
        SELECT Person2Id
        FROM Person_knows_Person
        WHERE Person1Id = 13194139540984
        UNION ALL
        SELECT k2.Person2Id
        FROM Person_knows_Person k1, Person_knows_Person k2
        WHERE k1.Person1Id = 13194139540984
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> 13194139540984
        UNION ALL
            SELECT k3.Person2Id
        FROM Person_knows_Person k1, Person_knows_Person k2, Person_knows_Person k3
        WHERE k1.Person1Id = 13194139540984
          AND k2.Person1Id = k1.Person2Id
          AND k2.Person2Id = k3.Person1Id
          AND k2.Person2Id <> 13194139540984
          AND k3.Person2Id != 13194139540984
"""
)

In [ ]:
len(all_friends.fetchall())

In [ ]:
id_check = cursor.execute("""
        SELECT k2.Person2Id
        FROM Person_knows_Person k1, Person_knows_Person k2
        WHERE k1.Person1Id = 9552
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> 9552
          
""").fetch_df()
len(id_check)

In [ ]:
second_hop = cursor.execute("""

        SELECT personId, frequency, count(k2.Person2Id) as second_hop_count
        FROM Person_knows_Person k1, Person_knows_Person k2, 
            (SELECT
        id AS personId,
        frequency,
        abs(frequency - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    ORDER BY diff, md5(id)
    LIMIT 150
    ) x
        
        WHERE k1.Person1Id = x.personId
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> x.personId
        GROUP BY x.personId, frequency
        ORDER BY frequency,second_hop_count DESC
""").fetch_df()

In [ ]:
second_hop.head(100)

In [ ]:
#count(k2.Person2Id)
#abs(count(k2.Person2Id) - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY count(k2.Person2Id)))) AS diff2

In [ ]:
third_hop = cursor.execute("""
;WITH selected_ids AS (
SELECT
    id AS personId,
    frequency,
    abs(frequency - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    ORDER BY diff, md5(id)
    LIMIT 150
)

,second_hop_selected AS (
    SELECT personId, count(k2.Person2Id) as second_hop_count
        FROM Person_knows_Person k1,
             Person_knows_Person k2, 
             selected_ids
        WHERE k1.Person1Id = personId
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> personId
        GROUP BY personId

)
, third_hop_selected AS (
        SELECT personId, count(k3.Person2Id) as third_hop_count
        FROM Person_knows_Person k1, Person_knows_Person k2, Person_knows_Person k3, selected_ids
        WHERE k1.Person1Id = personId
          AND k2.Person1Id = k1.Person2Id
          AND k2.Person2Id = k3.Person1Id
          AND k2.Person2Id <> personId
          AND k3.Person2Id != personId
          GROUP BY personId
)

SELECT selected_ids.personId, frequency, second_hop_count, third_hop_count
FROM selected_ids, second_hop_selected, third_hop_selected
WHERE selected_ids.personId = second_hop_selected.personId AND selected_ids.personId = third_hop_selected.personId
ORDER BY frequency, second_hop_count, third_hop_count DESC
""").fetch_df()

In [ ]:
third_hop.head(100)

In [ ]:
third_hop = cursor.execute("""
;WITH selected_ids AS (
SELECT
    id AS personId,
    frequency,
    abs(frequency - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    ORDER BY diff, md5(id)
    LIMIT 500
)

,second_hop_selected AS (
    SELECT personId, k2.Person2Id fofids, count(k2.Person2Id) as second_hop_count
        FROM Person_knows_Person k1,
             Person_knows_Person k2, 
             selected_ids
        WHERE k1.Person1Id = personId
          AND k1.Person2Id = k2.Person1Id
          AND k2.Person2Id <> personId
        GROUP BY personId

)
, selected_ids_2 AS (
    SELECT personId, second_hop_count,
    abs(second_hop_count - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY second_hop_count) FROM second_hop_selected)) AS diff2
        FROM second_hop_selected
    ORDER BY diff2, md5(id)
    LIMIT 150
)


, third_hop_selected AS (
        SELECT personId, count(k3.Person2Id) as third_hop_count
        FROM Person_knows_Person k1, Person_knows_Person k2, Person_knows_Person k3, selected_ids_2
        WHERE k1.Person1Id = personId
          AND k2.Person1Id = k1.Person2Id
          AND k2.Person2Id = k3.Person1Id
          AND k2.Person2Id <> personId
          AND k3.Person2Id != personId
          GROUP BY personId
)

SELECT selected_ids.personId, frequency, second_hop_count, third_hop_count
FROM selected_ids, second_hop_selected, third_hop_selected
WHERE selected_ids.personId = second_hop_selected.personId AND selected_ids.personId = third_hop_selected.personId
ORDER BY frequency, second_hop_count, third_hop_count DESC
""").fetch_df()

In [ ]:
second_hop_curated = cursor.execute("""
-- WIP: This now tries to minimize the difference between frequency of friends and friend of friends. TODO: third hop
;WITH selected_ids AS (
    SELECT id AS personId,
           frequency,
           abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
      FROM personNumFriends
     ORDER BY diff, md5(id)
     LIMIT 1000
)
, second_hop_selected AS (
    SELECT personId, sum(personNumFriends.frequency) AS second_hop_count
      FROM personNumFriends, 
           Person_knows_Person k1,
           selected_ids
     WHERE k1.Person1Id = selected_ids.personId
       AND personNumFriends.id = k1.person2Id
     GROUP BY personId
)
, selected_ids_2 AS (
    SELECT personId,
           second_hop_count as frequency,
           abs(second_hop_count - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY second_hop_count) FROM second_hop_selected)) AS diff2
      FROM second_hop_selected
     ORDER BY diff2, md5(personId)
     LIMIT 100
)

SELECT
    personId AS 'personId',
    firstName AS 'firstName'
FROM
    (
        SELECT selected_ids.personId
        FROM selected_ids, selected_ids_2
        WHERE selected_ids.personId = selected_ids_2.personId
        ORDER BY diff2, md5(selected_ids.personId)
        LIMIT 50
    ),
    (SELECT
        firstName,
        abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM personFirstNames)) AS diff
    FROM personFirstNames
    ORDER BY diff, md5(firstName)
    LIMIT 20
    )
ORDER BY md5(concat(personId, firstName))
LIMIT 500


""").fetch_df()

In [ ]:
second_hop_curated.head(500)

In [ ]:
second_hop_curated_new = cursor.execute("""
;WITH selected_ids AS (
SELECT
    id AS personId,
    frequency,
    abs(frequency - (SELECT percentile_disc(0.65) WITHIN GROUP (ORDER BY frequency) FROM personNumFriends)) AS diff
    FROM personNumFriends
    ORDER BY diff, md5(id)
)

,second_hop_selected AS (
    SELECT personId, sum(personNumFriends.frequency) AS second_hop_count
        FROM personNumFriends, 
             Person_knows_Person k1,
             selected_ids
        WHERE k1.Person1Id = selected_ids.personId
           AND personNumFriends.id = k1.person2Id
        GROUP BY personId
)
, selected_ids_2 AS (
    SELECT personId, second_hop_count as frequency,
    abs(second_hop_count - (SELECT percentile_disc(0.55) WITHIN GROUP (ORDER BY second_hop_count) FROM second_hop_selected)) AS diff2
        FROM second_hop_selected
    ORDER BY diff2, md5(personId)
    LIMIT 100
)

SELECT selected_ids.personId, selected_ids.frequency, selected_ids_2.frequency - selected_ids.frequency as frequency_2, diff2
FROM selected_ids, selected_ids_2
WHERE selected_ids.personId = selected_ids_2.personId
ORDER BY diff2
LIMIT 50
""").fetch_df()

In [ ]:
second_hop_curated_new.head(50)

In [ ]:
people4hops = 'people2Hops'
people4hops_path = f"factors/{people4hops}/*.parquet"


# cursor.execute(f"DROP VIEW {people4hops};")
cursor.execute(f"CREATE VIEW {people4hops} AS SELECT * FROM read_parquet('{people4hops_path}');")
cursor.execute(f"SELECT * FROM {people4hops} LIMIT 10").fetch_df().head()